In [1]:
# Import my own classes
import Prot_Post as Post
import OCTP_postprocess_CLASS as octp

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import binned_statistic_2d
from typing import Tuple

from IPython.core.getipython import get_ipython

def set_plot(inline=True, tex=False):
    ipython = get_ipython()
    if inline:
        """Set the Matplotlib backend to inline for Jupyter Notebook."""
        if ipython is not None:
            ipython.run_line_magic("matplotlib", "inline")
        Post.set_plot_settings(tex=tex, svg=False)
        print("Switched to Inline Renderer")
    
    else:
        """Set the Matplotlib backend to SVG for saving high-quality images."""
        Post.set_plot_settings(tex=tex, svg=True)
        print("Switched to SVG Renderer")

In [2]:
set_plot(inline=True, tex=False)

Switched to Inline Renderer


### LOAD THE SAVED DATA
For this part, we will only incude the 15 C particles.

In [3]:
runs = [r"./run_1/", r"./run_2/", r"./run_3/", r"./run_4/", r"./run_5/", r"./run_6/"]
T = 15
folders = [r"Temp_"+str(T)+"/"]
base_dir = r"MLMD/"
posts=[]

for run in runs:
    posts.append(Post.Prot_Post(base_dir+folders[0]+run))

no automatic filepath conversiona evailable use relative path
no automatic filepath conversiona evailable use relative path
no automatic filepath conversiona evailable use relative path
no automatic filepath conversiona evailable use relative path
no automatic filepath conversiona evailable use relative path
no automatic filepath conversiona evailable use relative path


### LOAD MLMD INFORMATION

In [4]:
F_index=1
r_bond, theta_bond, hyd_n_OH, hyd_n_K, hyd_n_w, hyd_n_OH_r, hyd_n_K_r, hyd_n_w_r = np.zeros(len(posts)), np.zeros(len(posts)), np.zeros(len(posts)), np.zeros(len(posts)), np.zeros(len(posts)), np.zeros(len(posts)), np.zeros(len(posts)), np.zeros(len(posts))
rdf_HOH, rdf_HH2O, rdf_HH, rdf_H2OH2O, rdf_OHH2O, rdf_KH2O = [], [], [], [], [], []
rdf_F_HOH, rdf_F_HH2O, rdf_F_HH, rdf_F_H2OH2O, rdf_F_OHH2O, rdf_F_KH2O = [], [], [], [], [], []
n_F_H2OH2O, n_F_OHH2O, n_F_KH2O = [], [], []
T = 15  # retrieve average temperature T is in Celcius, t in Kelvin

for i, post in enumerate(posts):
    # Structure, traditional
    # intra molecular
    rdf_HOH.append(post.rdf_HOH)
    rdf_HH2O.append(post.rdf_HH2O)
    rdf_HH.append(post.rdf_HH)
    
    # inter molecular
    rdf_OHH2O.append(post.rdf_OHH2O)
    rdf_KH2O.append(post.rdf_KH2O)
    rdf_H2OH2O.append(post.rdf_H2OH2O)
    rdf_r = post.rdf_r
    
    # Structure, Force
    # intra molecular
    r_bond[i], theta_bond[i] = post.water_shape()
    rdf_F_HOH.append(post.rdf_F_HOH[F_index, :])
    rdf_F_HH2O.append(post.rdf_F_HH2O[F_index, :])
    rdf_F_HH.append(post.rdf_F_HH[F_index, :])
    
    # inter molecular
    rdf_F_OHH2O.append(post.rdf_F_OHH2O[F_index, :])
    rdf_F_KH2O.append(post.rdf_F_KH2O[F_index, :])
    rdf_F_H2OH2O.append(post.rdf_F_H2OH2O[F_index, :])
    rdf_F_r = post.rdf_F_r
    
    # coordination number
    N_hydr = post.cordination_N('OH', 'H2O', force_rdf=True, F_idx=F_index)
    n_F_OHH2O.append(N_hydr)
    hyd_n_OH_r[i] = rdf_F_r[np.argmin(np.abs(rdf_F_r-3))+np.argmin(rdf_F_OHH2O[i][np.argmin(np.abs(rdf_F_r-3)):np.argmin(np.abs(rdf_F_r-4.5))])]
    hyd_n_OH[i] = N_hydr[np.argmin(np.abs(rdf_F_r-3))+np.argmin(rdf_F_OHH2O[i][np.argmin(np.abs(rdf_F_r-3)):np.argmin(np.abs(rdf_F_r-4.5))])]

    N_hydr = post.cordination_N('K', 'H2O', force_rdf=True, F_idx=F_index)
    n_F_KH2O.append(N_hydr)
    hyd_n_K_r[i] = rdf_F_r[np.argmin(np.abs(rdf_F_r-3))+np.argmin(rdf_F_KH2O[i][np.argmin(np.abs(rdf_F_r-3)):np.argmin(np.abs(rdf_F_r-4.5))])]
    hyd_n_K[i] = N_hydr[np.argmin(np.abs(rdf_F_r-3))+np.argmin(rdf_F_KH2O[i][np.argmin(np.abs(rdf_F_r-3)):np.argmin(np.abs(rdf_F_r-4.5))])]
    
    N_hydr = post.cordination_N('H2O', 'H2O', force_rdf=True, F_idx=F_index)
    n_F_H2OH2O.append(N_hydr)
    hyd_n_w_r[i] = rdf_F_r[np.argmin(np.abs(rdf_F_r-3))+np.argmin(rdf_F_H2OH2O[i][np.argmin(np.abs(rdf_F_r-3)):np.argmin(np.abs(rdf_F_r-4.5))])]
    hyd_n_w[i] = N_hydr[np.argmin(np.abs(rdf_F_r-3))+np.argmin(rdf_F_H2OH2O[i][np.argmin(np.abs(rdf_F_r-3)):np.argmin(np.abs(rdf_F_r-4.5))])]

    

In [5]:
# Structure, traditional
# intra molecular
rdf_HOH_ave, rdf_HOH_err = Post.averages(np.array(rdf_HOH))
rdf_HH2O_ave, rdf_HH2O_err = Post.averages(np.array(rdf_HH2O))
rdf_HH_ave, rdf_HH_err = Post.averages(np.array(rdf_HH))
# inter molecular
rdf_OHH2O_ave, rdf_OHH2O_err = Post.averages(np.array(rdf_OHH2O))
rdf_KH2O_ave, rdf_KH2O_err = Post.averages(np.array(rdf_KH2O))
rdf_H2OH2O_ave, rdf_H2OH2O_err = Post.averages(np.array(rdf_H2OH2O))

# Structure, Force
# intra molecular
r_bond_ave, r_bond_err = Post.averages(r_bond)
theta_bond_ave, theta_bond_err = Post.averages(theta_bond)
rdf_F_HOH_ave, rdf_F_HOH_err = Post.averages(np.array(rdf_F_HOH))
rdf_F_HH2O_ave, rdf_F_HH2O_err = Post.averages(np.array(rdf_F_HH2O))
rdf_F_HH_ave, rdf_F_HH_err = Post.averages(np.array(rdf_F_HH))
# inter molecular
rdf_F_OHH2O_ave, rdf_F_OHH2O_err = Post.averages(np.array(rdf_F_OHH2O))
rdf_F_KH2O_ave, rdf_F_KH2O_err = Post.averages(np.array(rdf_F_KH2O))
rdf_F_H2OH2O_ave, rdf_F_H2OH2O_err = Post.averages(np.array(rdf_F_H2OH2O))

n_F_OHH2O_ave, n_F_OHH2O_err = Post.averages(np.array(n_F_OHH2O))
n_F_KH2O_ave, n_F_KH2O_err = Post.averages(np.array(n_F_KH2O))
n_F_H2OH2O_ave, n_F_H2OH2O_err = Post.averages(np.array(n_F_H2OH2O))

# Hydration numbers
hyd_n_OH_ave, hyd_n_OH_err = Post.averages(hyd_n_OH)
hyd_n_K_ave, hyd_n_K_err = Post.averages(hyd_n_K)
hyd_n_w_ave, hyd_n_w_err = Post.averages(hyd_n_w)

hyd_n_OH_r_ave, hyd_n_OH_r_err = Post.averages(hyd_n_OH_r)
hyd_n_K_r_ave, hyd_n_K_r_err = Post.averages(hyd_n_K_r)
hyd_n_w_r_ave, hyd_n_w_r_err = Post.averages(hyd_n_w_r)

In [6]:
print("Structure properties")
print(f'The water bond length is {r_bond} Angstrom')
print(f'The water angle is {theta_bond} degree')
print(f'The hydration number OH- is {hyd_n_OH}')
print(f'The hydration number K+ is {hyd_n_K}')
print(f'The hydration number water is {hyd_n_w}')

Structure properties
The water bond length is [0.9767029 0.9767029 0.9767029 0.9767029 0.9767029 0.9767029] Angstrom
The water angle is [104.81415214 104.81415214 104.81415214 104.81415214 104.81415214
 104.81415214] degree
The hydration number OH- is [5.38464822 5.8624642  5.75675162 5.50173453 5.13806354 5.41417134]
The hydration number K+ is [7.70490781 7.33486242 6.744429   7.42233482 7.48956051 7.15610783]
The hydration number water is [12.62730699 12.71587109 12.68382003 12.68718959 12.6118053  12.66187219]


In [7]:
print("Structure properties")
print(f'The water bond length is ({r_bond_ave} ± {r_bond_err}) Angstrom')
print(f'angle is ({theta_bond_ave} ± {theta_bond_err}) degree')
print(f'The hydration number OH- is {hyd_n_OH_ave} ± {hyd_n_OH_err} at {hyd_n_OH_r_ave} ± {hyd_n_OH_r_err} Angstrom')
print(f'The hydration number K+ is {hyd_n_K_ave} ± {hyd_n_K_err} at {hyd_n_K_r_ave} ± {hyd_n_K_r_err} Angstrom')
print(f'The hydration number water is {hyd_n_w_ave} ± {hyd_n_w_err} at {hyd_n_w_r_ave} ± {hyd_n_w_r_err} Angstrom')

Structure properties
The water bond length is (0.9767029024658204 ± 9.731355061510588e-17) Angstrom
angle is (104.81415214176214 ± 1.2456134478733553e-14) degree
The hydration number OH- is 5.509638907365445 ± 0.21122127274074676 at 3.4648399053955075 ± 0.018500409295398448 Angstrom
The hydration number K+ is 7.30870040007894 ± 0.26416520119418924 at 3.698860588419596 ± 0.031920030024290294 Angstrom
The hydration number water is 12.664644198557566 ± 0.03138779364356695 at 4.497926029418945 ± 0.0 Angstrom


## HEAVY water 10u per H

In [8]:
runs = [r"./run_1/", r"./run_2/", r"./run_3/", r"./run_4/", r"./run_5/", r"./run_6/"]
T = 15
folders = [r"Temp_"+str(T)+"/"]
base_dir = r"MLMD_heavy/"
posts_heavy=[]

for run in runs:
    posts_heavy.append(Post.Prot_Post(base_dir+folders[0]+run+"part_uH_10"))

no automatic filepath conversiona evailable use relative path
no automatic filepath conversiona evailable use relative path
no automatic filepath conversiona evailable use relative path
no automatic filepath conversiona evailable use relative path
no automatic filepath conversiona evailable use relative path
no automatic filepath conversiona evailable use relative path


In [9]:
F_index=0
MLMD_heavy_r_bond, MLMD_heavy_theta_bond, MLMD_heavy_hyd_n_OH, MLMD_heavy_hyd_n_K, MLMD_heavy_hyd_n_w, MLMD_heavy_hyd_n_OH_r, MLMD_heavy_hyd_n_K_r, MLMD_heavy_hyd_n_w_r = np.zeros(len(posts)), np.zeros(len(posts)), np.zeros(len(posts)), np.zeros(len(posts)), np.zeros(len(posts)), np.zeros(len(posts)), np.zeros(len(posts)), np.zeros(len(posts))
MLMD_heavy_rdf_HOH, MLMD_heavy_rdf_HH2O, MLMD_heavy_rdf_HH, MLMD_heavy_rdf_H2OH2O, MLMD_heavy_rdf_OHH2O, MLMD_heavy_rdf_KH2O = [], [], [], [], [], []
MLMD_heavy_rdf_F_HOH, MLMD_heavy_rdf_F_HH2O, MLMD_heavy_rdf_F_HH, MLMD_heavy_rdf_F_H2OH2O, MLMD_heavy_rdf_F_OHH2O, MLMD_heavy_rdf_F_KH2O = [], [], [], [], [], []
MLMD_heavy_n_F_H2OH2O, MLMD_heavy_n_F_OHH2O, MLMD_heavy_n_F_KH2O = [], [], []
T = 15  # retrieve average temperature T is in Celcius, t in Kelvin

for i, post in enumerate(posts_heavy):
    # Structure, traditional
    # intra molecular
    MLMD_heavy_rdf_HOH.append(post.rdf_HOH)
    MLMD_heavy_rdf_HH2O.append(post.rdf_HH2O)
    MLMD_heavy_rdf_HH.append(post.rdf_HH)
    
    # inter molecular
    MLMD_heavy_rdf_OHH2O.append(post.rdf_OHH2O)
    MLMD_heavy_rdf_KH2O.append(post.rdf_KH2O)
    MLMD_heavy_rdf_H2OH2O.append(post.rdf_H2OH2O)
    MLMD_heavy_rdf_r = post.rdf_r
    
    # Structure, Force
    # intra molecular
    MLMD_heavy_r_bond[i], MLMD_heavy_theta_bond[i] = post.water_shape()
    MLMD_heavy_rdf_F_HOH.append(post.rdf_F_HOH[F_index, :])
    MLMD_heavy_rdf_F_HH2O.append(post.rdf_F_HH2O[F_index, :])
    MLMD_heavy_rdf_F_HH.append(post.rdf_F_HH[F_index, :])
    
    # inter molecular
    MLMD_heavy_rdf_F_OHH2O.append(post.rdf_F_OHH2O[F_index, :])
    MLMD_heavy_rdf_F_KH2O.append(post.rdf_F_KH2O[F_index, :])
    MLMD_heavy_rdf_F_H2OH2O.append(post.rdf_F_H2OH2O[F_index, :])
    MLMD_heavy_rdf_F_r = post.rdf_F_r
    
    # coordination number
    MLMD_heavy_N_hydr = post.cordination_N('OH', 'H2O', force_rdf=True, F_idx=F_index)
    MLMD_heavy_n_F_OHH2O.append(MLMD_heavy_N_hydr)
    MLMD_heavy_hyd_n_OH_r[i] = MLMD_heavy_rdf_F_r[np.argmin(np.abs(MLMD_heavy_rdf_F_r-3))+np.argmin(MLMD_heavy_rdf_F_OHH2O[i][np.argmin(np.abs(MLMD_heavy_rdf_F_r-3)):np.argmin(np.abs(MLMD_heavy_rdf_F_r-4.5))])]
    MLMD_heavy_hyd_n_OH[i] = MLMD_heavy_N_hydr[np.argmin(np.abs(MLMD_heavy_rdf_F_r-3))+np.argmin(MLMD_heavy_rdf_F_OHH2O[i][np.argmin(np.abs(MLMD_heavy_rdf_F_r-3)):np.argmin(np.abs(MLMD_heavy_rdf_F_r-4.5))])]

    MLMD_heavy_N_hydr = post.cordination_N('K', 'H2O', force_rdf=True, F_idx=F_index)
    MLMD_heavy_n_F_KH2O.append(MLMD_heavy_N_hydr)
    MLMD_heavy_hyd_n_K_r[i] = MLMD_heavy_rdf_F_r[np.argmin(np.abs(MLMD_heavy_rdf_F_r-3))+np.argmin(MLMD_heavy_rdf_F_KH2O[i][np.argmin(np.abs(MLMD_heavy_rdf_F_r-3)):np.argmin(np.abs(MLMD_heavy_rdf_F_r-4.5))])]
    MLMD_heavy_hyd_n_K[i] = MLMD_heavy_N_hydr[np.argmin(np.abs(MLMD_heavy_rdf_F_r-3))+np.argmin(MLMD_heavy_rdf_F_KH2O[i][np.argmin(np.abs(MLMD_heavy_rdf_F_r-3)):np.argmin(np.abs(MLMD_heavy_rdf_F_r-4.5))])]
    
    MLMD_heavy_N_hydr = post.cordination_N('H2O', 'H2O', force_rdf=True, F_idx=F_index)
    MLMD_heavy_n_F_H2OH2O.append(MLMD_heavy_N_hydr)
    MLMD_heavy_hyd_n_w_r[i] = MLMD_heavy_rdf_F_r[np.argmin(np.abs(MLMD_heavy_rdf_F_r-3))+np.argmin(MLMD_heavy_rdf_F_H2OH2O[i][np.argmin(np.abs(MLMD_heavy_rdf_F_r-3)):np.argmin(np.abs(MLMD_heavy_rdf_F_r-4.5))])]
    MLMD_heavy_hyd_n_w[i] = MLMD_heavy_N_hydr[np.argmin(np.abs(MLMD_heavy_rdf_F_r-3))+np.argmin(MLMD_heavy_rdf_F_H2OH2O[i][np.argmin(np.abs(MLMD_heavy_rdf_F_r-3)):np.argmin(np.abs(MLMD_heavy_rdf_F_r-4.5))])]


In [10]:
# Structure, traditional
# intra molecular
MLMD_heavy_rdf_HOH_ave, MLMD_heavy_rdf_HOH_err = Post.averages(np.array(MLMD_heavy_rdf_HOH))
MLMD_heavy_rdf_HH2O_ave, MLMD_heavy_rdf_HH2O_err = Post.averages(np.array(MLMD_heavy_rdf_HH2O))
MLMD_heavy_rdf_HH_ave, MLMD_heavy_rdf_HH_err = Post.averages(np.array(MLMD_heavy_rdf_HH))
# inter molecular
MLMD_heavy_rdf_OHH2O_ave, MLMD_heavy_rdf_OHH2O_err = Post.averages(np.array(MLMD_heavy_rdf_OHH2O))
MLMD_heavy_rdf_KH2O_ave, MLMD_heavy_rdf_KH2O_err = Post.averages(np.array(MLMD_heavy_rdf_KH2O))
MLMD_heavy_rdf_H2OH2O_ave, MLMD_heavy_rdf_H2OH2O_err = Post.averages(np.array(MLMD_heavy_rdf_H2OH2O))

# Structure, Force
# intra molecular
MLMD_heavy_r_bond_ave, MLMD_heavy_r_bond_err = Post.averages(MLMD_heavy_r_bond)
MLMD_heavy_theta_bond_ave, MLMD_heavy_theta_bond_err = Post.averages(MLMD_heavy_theta_bond)
MLMD_heavy_rdf_F_HOH_ave, MLMD_heavy_rdf_F_HOH_err = Post.averages(np.array(MLMD_heavy_rdf_F_HOH))
MLMD_heavy_rdf_F_HH2O_ave, MLMD_heavy_rdf_F_HH2O_err = Post.averages(np.array(MLMD_heavy_rdf_F_HH2O))
MLMD_heavy_rdf_F_HH_ave, MLMD_heavy_rdf_F_HH_err = Post.averages(np.array(MLMD_heavy_rdf_F_HH))
# inter molecular
MLMD_heavy_rdf_F_OHH2O_ave, MLMD_heavy_rdf_F_OHH2O_err = Post.averages(np.array(MLMD_heavy_rdf_F_OHH2O))
MLMD_heavy_rdf_F_KH2O_ave, MLMD_heavy_rdf_F_KH2O_err = Post.averages(np.array(MLMD_heavy_rdf_F_KH2O))
MLMD_heavy_rdf_F_H2OH2O_ave, MLMD_heavy_rdf_F_H2OH2O_err = Post.averages(np.array(MLMD_heavy_rdf_F_H2OH2O))

MLMD_heavy_n_F_OHH2O_ave, MLMD_heavy_n_F_OHH2O_err = Post.averages(np.array(MLMD_heavy_n_F_OHH2O))
MLMD_heavy_n_F_KH2O_ave, MLMD_heavy_n_F_KH2O_err = Post.averages(np.array(MLMD_heavy_n_F_KH2O))
MLMD_heavy_n_F_H2OH2O_ave, MLMD_heavy_n_F_H2OH2O_err = Post.averages(np.array(MLMD_heavy_n_F_H2OH2O))

# Hydration numbers
MLMD_heavy_hyd_n_OH_ave, MLMD_heavy_hyd_n_OH_err = Post.averages(MLMD_heavy_hyd_n_OH)
MLMD_heavy_hyd_n_K_ave, MLMD_heavy_hyd_n_K_err = Post.averages(MLMD_heavy_hyd_n_K)
MLMD_heavy_hyd_n_w_ave, MLMD_heavy_hyd_n_w_err = Post.averages(MLMD_heavy_hyd_n_w)

MLMD_heavy_hyd_n_OH_r_ave, MLMD_heavy_hyd_n_OH_r_err = Post.averages(MLMD_heavy_hyd_n_OH_r)
MLMD_heavy_hyd_n_K_r_ave, MLMD_heavy_hyd_n_K_r_err = Post.averages(MLMD_heavy_hyd_n_K_r)
MLMD_heavy_hyd_n_w_r_ave, MLMD_heavy_hyd_n_w_r_err = Post.averages(MLMD_heavy_hyd_n_w_r)

In [11]:
print("Structure properties")
print(f'The water bond length is {MLMD_heavy_r_bond} Angstrom')
print(f'The water angle is {MLMD_heavy_theta_bond} degree')
print(f'The hydration number OH- is {MLMD_heavy_hyd_n_OH}')
print(f'The hydration number K+ is {MLMD_heavy_hyd_n_K}')
print(f'The hydration number water is {MLMD_heavy_hyd_n_w}')

print("Structure properties")
print(f'The water bond length is ({MLMD_heavy_r_bond_ave} ± {MLMD_heavy_r_bond_err}) Angstrom')
print(f'angle is ({MLMD_heavy_theta_bond_ave} ± {MLMD_heavy_theta_bond_err}) degree')
print(f'The hydration number OH- is {MLMD_heavy_hyd_n_OH_ave} ± {MLMD_heavy_hyd_n_OH_err} at {MLMD_heavy_hyd_n_OH_r_ave} ± {MLMD_heavy_hyd_n_OH_r_err} Angstrom')
print(f'The hydration number K+ is {MLMD_heavy_hyd_n_K_ave} ± {MLMD_heavy_hyd_n_K_err} at {MLMD_heavy_hyd_n_K_r_ave} ± {MLMD_heavy_hyd_n_K_r_err} Angstrom')
print(f'The hydration number water is {MLMD_heavy_hyd_n_w_ave} ± {MLMD_heavy_hyd_n_w_err} at {MLMD_heavy_hyd_n_w_r_ave} ± {MLMD_heavy_hyd_n_w_r_err} Angstrom')

Structure properties
The water bond length is [0.9767029 0.9767029 0.9767029 0.9767029 0.9767029 0.9767029] Angstrom
The water angle is [104.81415214 104.81415214 104.81415214 104.81415214 104.81415214
 104.81415214] degree
The hydration number OH- is [5.43817797 5.64067299 5.44028848 5.93449658 5.33125431 5.08418916]
The hydration number K+ is [7.46066177 7.34178455 7.6353726  6.96833227 7.66877928 7.36759122]
The hydration number water is [12.58811571 12.57025801 12.6377677  12.64003867 12.59794901 12.57200139]
Structure properties
The water bond length is (0.9767029024658204 ± 9.731355061510588e-17) Angstrom
angle is (104.81415214176214 ± 1.2456134478733553e-14) degree
The hydration number OH- is 5.478179912891441 ± 0.23037725469238365 at 3.4854531261800132 ± 0.03707706185852151 Angstrom
The hydration number K+ is 7.407086946922095 ± 0.20299192894667975 at 3.6855226220296227 ± 0.014099910511749162 Angstrom
The hydration number water is 12.601021748606199 ± 0.024887637043979782 at 4.

## Import AIMD simulations

In [12]:
runs = [r"./run_1/", r"./run_2/", r"./run_3/", r"./run_4/", r"./run_5/"] #, r"./run6/"]
folders = [r"Temp_"+str(T)+"/"]
base_dir = r"AIMD/AIMD_1m_2/"
posts=[]

for run in runs:
    posts.append(Post.Prot_Post(base_dir+run))

no automatic filepath conversiona evailable use relative path
no automatic filepath conversiona evailable use relative path
no automatic filepath conversiona evailable use relative path
no automatic filepath conversiona evailable use relative path
no automatic filepath conversiona evailable use relative path


In [13]:
F_index=0
AIMD_r_bond, AIMD_theta_bond, AIMD_hyd_n_OH, AIMD_hyd_n_K, AIMD_hyd_n_w, AIMD_hyd_n_OH_r, AIMD_hyd_n_K_r, AIMD_hyd_n_w_r = np.zeros(len(posts)), np.zeros(len(posts)), np.zeros(len(posts)), np.zeros(len(posts)), np.zeros(len(posts)), np.zeros(len(posts)), np.zeros(len(posts)), np.zeros(len(posts))
AIMD_rdf_HOH, AIMD_rdf_HH2O, AIMD_rdf_HH, AIMD_rdf_H2OH2O, AIMD_rdf_OHH2O, AIMD_rdf_KH2O = [], [], [], [], [], []
AIMD_rdf_F_HOH, AIMD_rdf_F_HH2O, AIMD_rdf_F_HH, AIMD_rdf_F_H2OH2O, AIMD_rdf_F_OHH2O, AIMD_rdf_F_KH2O = [], [], [], [], [], []
AIMD_n_F_H2OH2O, AIMD_n_F_OHH2O, AIMD_n_F_KH2O = [], [], []
T = 15  # retrieve average temperature T is in Celcius, t in Kelvin

for i, post in enumerate(posts):
    # Structure, traditional
    # intra molecular
    AIMD_rdf_HOH.append(post.rdf_HOH)
    AIMD_rdf_HH2O.append(post.rdf_HH2O)
    AIMD_rdf_HH.append(post.rdf_HH)
    
    # inter molecular
    AIMD_rdf_OHH2O.append(post.rdf_OHH2O)
    AIMD_rdf_KH2O.append(post.rdf_KH2O)
    AIMD_rdf_H2OH2O.append(post.rdf_H2OH2O)
    AIMD_rdf_r = post.rdf_r
    
    # Structure, Force
    # intra molecular
    AIMD_r_bond[i], AIMD_theta_bond[i] = post.water_shape()
    AIMD_rdf_F_HOH.append(post.rdf_F_HOH[F_index, :])
    AIMD_rdf_F_HH2O.append(post.rdf_F_HH2O[F_index, :])
    AIMD_rdf_F_HH.append(post.rdf_F_HH[F_index, :])
    
    # inter molecular
    AIMD_rdf_F_OHH2O.append(post.rdf_F_OHH2O[F_index, :])
    AIMD_rdf_F_KH2O.append(post.rdf_F_KH2O[F_index, :])
    AIMD_rdf_F_H2OH2O.append(post.rdf_F_H2OH2O[F_index, :])
    AIMD_rdf_F_r = post.rdf_F_r
    
    # coordination number
    AIMD_N_hydr = post.cordination_N('OH', 'H2O', force_rdf=True, F_idx=F_index)
    AIMD_n_F_OHH2O.append(AIMD_N_hydr)
    AIMD_hyd_n_OH_r[i] = AIMD_rdf_F_r[np.argmin(np.abs(AIMD_rdf_F_r-3))+np.argmin(AIMD_rdf_F_OHH2O[i][np.argmin(np.abs(AIMD_rdf_F_r-3)):np.argmin(np.abs(AIMD_rdf_F_r-4.5))])]
    AIMD_hyd_n_OH[i] = AIMD_N_hydr[np.argmin(np.abs(AIMD_rdf_F_r-3))+np.argmin(AIMD_rdf_F_OHH2O[i][np.argmin(np.abs(AIMD_rdf_F_r-3)):np.argmin(np.abs(AIMD_rdf_F_r-4.5))])]

    AIMD_N_hydr = post.cordination_N('K', 'H2O', force_rdf=True, F_idx=F_index)
    AIMD_n_F_KH2O.append(AIMD_N_hydr)
    AIMD_hyd_n_K_r[i] = AIMD_rdf_F_r[np.argmin(np.abs(AIMD_rdf_F_r-3))+np.argmin(AIMD_rdf_F_KH2O[i][np.argmin(np.abs(AIMD_rdf_F_r-3)):np.argmin(np.abs(AIMD_rdf_F_r-4.5))])]
    AIMD_hyd_n_K[i] = AIMD_N_hydr[np.argmin(np.abs(AIMD_rdf_F_r-3))+np.argmin(AIMD_rdf_F_KH2O[i][np.argmin(np.abs(AIMD_rdf_F_r-3)):np.argmin(np.abs(AIMD_rdf_F_r-4.5))])]
    
    AIMD_N_hydr = post.cordination_N('H2O', 'H2O', force_rdf=True, F_idx=F_index)
    AIMD_n_F_H2OH2O.append(AIMD_N_hydr)
    AIMD_hyd_n_w_r[i] = AIMD_rdf_F_r[np.argmin(np.abs(AIMD_rdf_F_r-3))+np.argmin(AIMD_rdf_F_H2OH2O[i][np.argmin(np.abs(AIMD_rdf_F_r-3)):np.argmin(np.abs(AIMD_rdf_F_r-4.5))])]
    AIMD_hyd_n_w[i] = AIMD_N_hydr[np.argmin(np.abs(AIMD_rdf_F_r-3))+np.argmin(AIMD_rdf_F_H2OH2O[i][np.argmin(np.abs(AIMD_rdf_F_r-3)):np.argmin(np.abs(AIMD_rdf_F_r-4.5))])]


In [14]:
# Structure, traditional
# intra molecular
AIMD_rdf_HOH_ave, AIMD_rdf_HOH_err = Post.averages(np.array(AIMD_rdf_HOH))
AIMD_rdf_HH2O_ave, AIMD_rdf_HH2O_err = Post.averages(np.array(AIMD_rdf_HH2O))
AIMD_rdf_HH_ave, AIMD_rdf_HH_err = Post.averages(np.array(AIMD_rdf_HH))
# inter molecular
AIMD_rdf_OHH2O_ave, AIMD_rdf_OHH2O_err = Post.averages(np.array(AIMD_rdf_OHH2O))
AIMD_rdf_KH2O_ave, AIMD_rdf_KH2O_err = Post.averages(np.array(AIMD_rdf_KH2O))
AIMD_rdf_H2OH2O_ave, AIMD_rdf_H2OH2O_err = Post.averages(np.array(AIMD_rdf_H2OH2O))

# Structure, Force
# intra molecular
AIMD_r_bond_ave, AIMD_r_bond_err = Post.averages(AIMD_r_bond)
AIMD_theta_bond_ave, AIMD_theta_bond_err = Post.averages(AIMD_theta_bond)
AIMD_rdf_F_HOH_ave, AIMD_rdf_F_HOH_err = Post.averages(np.array(AIMD_rdf_F_HOH))
AIMD_rdf_F_HH2O_ave, AIMD_rdf_F_HH2O_err = Post.averages(np.array(AIMD_rdf_F_HH2O))
AIMD_rdf_F_HH_ave, AIMD_rdf_F_HH_err = Post.averages(np.array(AIMD_rdf_F_HH))
# inter molecular
AIMD_rdf_F_OHH2O_ave, AIMD_rdf_F_OHH2O_err = Post.averages(np.array(AIMD_rdf_F_OHH2O))
AIMD_rdf_F_KH2O_ave, AIMD_rdf_F_KH2O_err = Post.averages(np.array(AIMD_rdf_F_KH2O))
AIMD_rdf_F_H2OH2O_ave, AIMD_rdf_F_H2OH2O_err = Post.averages(np.array(AIMD_rdf_F_H2OH2O))

AIMD_n_F_OHH2O_ave, AIMD_n_F_OHH2O_err = Post.averages(np.array(AIMD_n_F_OHH2O))
AIMD_n_F_KH2O_ave, AIMD_n_F_KH2O_err = Post.averages(np.array(AIMD_n_F_KH2O))
AIMD_n_F_H2OH2O_ave, AIMD_n_F_H2OH2O_err = Post.averages(np.array(AIMD_n_F_H2OH2O))

# Hydration numbers
AIMD_hyd_n_OH_ave, AIMD_hyd_n_OH_err = Post.averages(AIMD_hyd_n_OH)
AIMD_hyd_n_K_ave, AIMD_hyd_n_K_err = Post.averages(AIMD_hyd_n_K)
AIMD_hyd_n_w_ave, AIMD_hyd_n_w_err = Post.averages(AIMD_hyd_n_w)

AIMD_hyd_n_OH_r_ave, AIMD_hyd_n_OH_r_err = Post.averages(AIMD_hyd_n_OH_r)
AIMD_hyd_n_K_r_ave, AIMD_hyd_n_K_r_err = Post.averages(AIMD_hyd_n_K_r)
AIMD_hyd_n_w_r_ave, AIMD_hyd_n_w_r_err = Post.averages(AIMD_hyd_n_w_r)

In [15]:
print("Structure properties")
print(f'The water bond length is {AIMD_r_bond} Angstrom')
print(f'The water angle is {AIMD_theta_bond} degree')
print(f'The hydration number OH- is {AIMD_hyd_n_OH}')
print(f'The hydration number K+ is {AIMD_hyd_n_K}')
print(f'The hydration number water is {AIMD_hyd_n_w}')

print("Structure properties")
print(f'The water bond length is ({AIMD_r_bond_ave} ± {AIMD_r_bond_err}) Angstrom')
print(f'angle is ({AIMD_theta_bond_ave} ± {AIMD_theta_bond_err}) degree')
print(f'The hydration number OH- is {AIMD_hyd_n_OH_ave} ± {AIMD_hyd_n_OH_err} at {AIMD_hyd_n_OH_r_ave} ± {AIMD_hyd_n_OH_r_err} Angstrom')
print(f'The hydration number K+ is {AIMD_hyd_n_K_ave} ± {AIMD_hyd_n_K_err} at {AIMD_hyd_n_K_r_ave} ± {AIMD_hyd_n_K_r_err} Angstrom')
print(f'The hydration number water is {AIMD_hyd_n_w_ave} ± {AIMD_hyd_n_w_err} at {AIMD_hyd_n_w_r_ave} ± {AIMD_hyd_n_w_r_err} Angstrom')

Structure properties
The water bond length is [0.98034053 0.98034053 0.98034053 0.98034053 0.98034053] Angstrom
The water angle is [104.95899186 104.95899186 104.95899186 104.95899186 104.61060957] degree
The hydration number OH- is [5.15861068 5.63283631 5.14773645 5.35508066 5.41328134]
The hydration number K+ is [7.62146352 7.57471241 8.03665932 8.21617244 7.77839479]
The hydration number water is [5.7350887  5.49063168 5.82570762 5.51840694 5.57604757]
Structure properties
The water bond length is (0.9803405296630858 ± 0.0) Angstrom
angle is (104.88931540485537 ± 0.13656334725306207) degree
The hydration number OH- is 5.341509087599956 ± 0.17596430587533926 at 3.3826295307373044 ± 0.023277958073908152 Angstrom
The hydration number K+ is 7.8454804955422635 ± 0.24082898174806008 at 3.712926080249024 ± 0.019706657043174366 Angstrom
The hydration number water is 5.62917650137334 ± 0.12711776832225347 at 3.5332272967041014 ± 0.023796269276334228 Angstrom


### IMPORT MD SIMULATIONS

In [16]:
folder = ['CMD/Temp_15']

for i in range(len(folder)):
    f_runs = ['run_1', 'run_2', 'run_3', 'run_4', 'run_5', 'run_6']  # All internal runs
    groups = ['wat', 'Oh', 'K']

    # Load the class
    mixture = octp.PP_OCTP(folder[i], f_runs, groups, dt=2, plotting=False)

    # Change the file names
    mixture.filenames(Diff_Onsag='diffonsag.dat',
                      T_conduc='tconductivity.dat',
                      Diff_self='diffself.dat')
    mixture.rdf()

### IMPORT DFT_MLFF force comparison

In [17]:
df = pd.read_csv('FF_accuracy_check/error_analysis/normal_13_final.csv')

### Plot structure properties

In [18]:
# Assuming you have your Post class and all necessary data loaded correctly
# Initialize subplots (adjust if you have a different setup)
fig, axs = plt.subplots(1, 3)

# colors
blue = "#1f77b4"
red = "#ff7f0e"

# PLOT PART 1
limits = [df['dft_f'].min(), df['dft_f'].max()]
string = '\n'.join((r'$R^2=%.4f$' % (df['cor_coef'][0], ), r'$RMSE=%.4f$ \si{\eV\per\angstrom}' % (df['rmse'][0], )))
axs[0].scatter(df['dft_f'], df['mlff_f'], alpha=0.15, edgecolors='none', s=2.5)
axs[0].text(0.05, 0.95, string, transform=axs[0].transAxes, fontsize=12, verticalalignment='top', bbox=dict(boxstyle='square', facecolor='white', alpha=0))
axs[0].set_xlim(limits)
axs[0].set_xlabel(r'MLFF Force/[\si{\eV\per\angstrom}]')
axs[0].set_ylim(limits)
axs[0].set_ylabel(r'DFT Force/[\si{\eV\per\angstrom}]')

# PLOT PART 2
# MLMD = BLUE - line
data = Post.line_with_errors(rdf_F_OHH2O_ave, rdf_F_OHH2O_err)
axs[1].plot(rdf_F_r, data[0], label='force rdf MLFF')
# AIMD = BLUE - scatter
data = Post.line_with_errors(AIMD_rdf_F_OHH2O_ave, AIMD_rdf_F_OHH2O_err)
axs[1].scatter(AIMD_rdf_r, AIMD_rdf_OHH2O_ave, marker='o', s=12, color=blue, label='rdf AIMD')
# CMD = BLUE - dashed
axs[1].plot(mixture.rdf_results['r/[Angstrom]'], mixture.rdf_results['g(r) watOh'],
            label=r'TIP4P+DFF/\ce{OH-}', color=blue, linestyle='-.', linewidth=1)

# Then the hydration number = RED
ax2 = axs[1].twinx()
ax2.plot(rdf_F_r[:-1], n_F_OHH2O_ave, color=red)
ax2.scatter(hyd_n_OH_r_ave, hyd_n_OH_ave, color=red, label='n_text{Oh-Ow}', marker='*')
ax2.set_ylim(0, 16)
ax2.set_ylabel('n_{Oh-Ow}(r)', color=red)
ax2.tick_params(axis='y', labelcolor=red, color=red)
ax2.yaxis.label.set_color(red)
ax2.spines['right'].set_color(red)

handles1, labels1 = axs[1].get_legend_handles_labels()
handles2, labels2 = ax2.get_legend_handles_labels()
axs[1].legend(handles1 + handles2, labels1 + labels2, loc='best')

axs[1].set_xlim(2, 7.5)
axs[1].set_xlabel(r'r/[\text{Å}]')
axs[1].set_ylim(0, 6)
axs[1].set_ylabel('g_{Oh-Ow}(r)', color=blue)
axs[1].yaxis.label.set_color(blue)
axs[1].tick_params(axis='y', labelcolor=blue, color=blue)
axs[1].spines['left'].set_color(blue)


# PLOT PART 3
# MLMD = BLUE - line
data = Post.line_with_errors(rdf_F_KH2O_ave, rdf_F_KH2O_err)
axs[2].plot(rdf_F_r, data[0], label='force rdf MLFF')
# AIMD = BLUE - scatter
data = Post.line_with_errors(AIMD_rdf_F_KH2O_ave, AIMD_rdf_F_KH2O_err)
axs[2].scatter(AIMD_rdf_r, AIMD_rdf_KH2O_ave, marker='o', s=12, color=blue, label='rdf AIMD')
# CMD = BLUE - dashed
axs[2].plot(mixture.rdf_results['r/[Angstrom]'], mixture.rdf_results['g(r) watK'],
            label=r'TIP4P+DFF/\ce{OH-}', color=blue, linestyle='-.', linewidth=1)

# Then the hydration number = RED
ax3 = axs[2].twinx()
ax3.plot(rdf_F_r[:-1], n_F_KH2O_ave, color=red)
ax3.scatter(hyd_n_K_r_ave, hyd_n_K_ave, color=red, label='n_{K-Ow}', marker='*')
ax3.set_ylim(0, 16)
ax3.set_ylabel('n_{K-Ow}(r)', color=red)
ax3.tick_params(axis='y', labelcolor=red, color=red)
ax3.yaxis.label.set_color(red)
ax3.spines['right'].set_color(red)

handles1, labels1 = axs[2].get_legend_handles_labels()
handles2, labels2 = ax3.get_legend_handles_labels()
axs[2].legend(handles1 + handles2, labels1 + labels2, loc='best')
axs[2].set_xlim(2, 7.5)
axs[2].set_xlabel(r'r/[\si{\angstrom}]')
axs[2].set_ylim(0, 6)
axs[2].set_ylabel('g_{K-Ow}(r)', color=blue)
axs[2].yaxis.label.set_color(blue)
axs[2].tick_params(axis='y', labelcolor=blue, color=blue)
axs[2].spines['left'].set_color(blue)


# Adjust layout to maintain aspect ratio
fig.tight_layout()
fig.savefig('plot1')
plt.close()

In [19]:
np.savetxt('plot1/1b1_rdf_OHH2O_mlff.txt', np.column_stack((rdf_F_r, rdf_F_OHH2O_ave, rdf_F_OHH2O_err)), delimiter=' ')
np.savetxt('plot1/1b2_rdf_OHH2O_aimd.txt', np.column_stack((AIMD_rdf_r, AIMD_rdf_OHH2O_ave, AIMD_rdf_OHH2O_err)), delimiter=' ')
np.savetxt('plot1/1b3_rdf_OHH2O_cmd.txt', np.column_stack((mixture.rdf_results['r/[Angstrom]'], mixture.rdf_results['g(r) watOh'])), delimiter=' ')
np.savetxt('plot1/1b4_hyd_run_OH_mlff.txt', np.column_stack((rdf_F_r[:-1], n_F_OHH2O_ave)), delimiter=' ')
np.savetxt('plot1/1b5_hyd_point_OH_mlff.txt', np.column_stack((hyd_n_OH_r_ave, hyd_n_OH_ave)), delimiter=' ')

np.savetxt('plot1/1c1_rdf_KH2O_mlff.txt', np.column_stack((rdf_F_r, rdf_F_KH2O_ave, rdf_F_KH2O_err)), delimiter='\t')
np.savetxt('plot1/1c2_rdf_KH2O_aimd.txt', np.column_stack((AIMD_rdf_r, AIMD_rdf_KH2O_ave, AIMD_rdf_KH2O_err)), delimiter=' ')
np.savetxt('plot1/1c3_rdf_KH2O_cmd.txt', np.column_stack((mixture.rdf_results['r/[Angstrom]'], mixture.rdf_results['g(r) watK'])), delimiter=' ')
np.savetxt('plot1/1c4_hyd_run_K_mlff.txt', np.column_stack((rdf_F_r[:-1], n_F_KH2O_ave)), delimiter=' ')
np.savetxt('plot1/1c5_hyd_point_K_mlff.txt', np.column_stack((hyd_n_K_r_ave, hyd_n_K_ave)), delimiter=' ')

### Tuckerman?

In [20]:
rdf_T_r = np.array([2.1087955814753747, 2.3794566838125664, 2.489416892386692,
                    2.5954641303676516, 2.7152883775509307, 2.8604873787949643,
                    2.9966958117539115, 3.21599862362252, 3.4387379677257126,
                    3.63596139084709, 3.827026407037171, 3.9926240283745518,
                    4.266324630980845, 4.520883881384495, 4.749768398020135,
                    4.956300014998986])

rdf_T= np.array([0.0213162050802449, 0.40471850433647116, 2.690676807157163,
                 4.055683292012599, 2.7466565497039914, 1.3489205142005103,
                 0.6446458033721258, 0.43986200933466346, 0.5568611535102042,
                 0.5850380709540239, 0.8684024316002157, 1.1239710254894524,
                 1.3298488631651386, 1.4635621707943294, 1.3087302917743804,
                 1.0428918043779394])

hyd_T_r = np.array([2.0706055178620275, 2.3952100299097414, 2.4910623693103116,
                    2.5360328566008774, 2.5873029177438, 2.6259208935866107,
                    2.70899762663114, 2.7921272972710676, 2.8942262729285964,
                    3.1045296935795523, 3.2702155442425953, 3.4677433585375113,
                    3.6144599041829526, 3.735669351779145, 3.8314952223820153,
                    3.9082635586416217, 3.9850054261035295])

hyd_T = np.array([-0.006507795060922916, 0.2161265561447312, 0.7600285863015159,
                  1.3149134029168623, 1.7921422962564302, 2.3581087161750824,
                  2.835408192975181, 3.3792819897477537, 3.7789537766562855,
                  4.256535587298506, 4.623061381141865, 5.034040638427401,
                  5.544768442134797, 5.977769739105884, 6.488384609276432,
                  7.032244289356897, 7.542816809451126])

## Save the parity plots

In [21]:
df = pd.read_csv('FF_accuracy_check/error_analysis/normal_13_final.csv')

np.savetxt('plot1/1a1_parity_long.txt', np.column_stack((df['dft_f'], df['mlff_f'] )), delimiter=' ')
np.savetxt('plot1/1a1_parity_short.txt', np.column_stack((df['dft_f'][:5000], df['mlff_f'][:5000] )), delimiter=' ')

### Plot for presentation

In [22]:
# Post.set_plot_settings(svg=True, tex=True)

# fig, axs = plt.subplots(figsize=(6, 4))

# # CMD
# tip4p_handle, = axs.plot(mixture.rdf_results['r/[Angstrom]'], mixture.rdf_results['g(r) watOh'], label=r'TIP4P+DFF/\ce{OH-}$^\text{4}$', color='C3')
# axs.set_xlim(2, 7.5)
# axs.set_xlabel(r'$r/[\text{Å}]$')
# axs.set_ylim(0, 6)
# axs.set_ylabel(r'rdf $g_{\text{O}_\text{h}\text{O}_\text{w}}(r)$')

# # Hydration number
# ax2 = axs.twinx()
# ax2.plot(rdf_F_r[:-1], n_F_OHH2O_ave, color=red)
# hydration_handle = ax2.scatter(hyd_n_OH_r_ave, hyd_n_OH_ave, color=red, label='$n_{Oh-Ow}$', marker='*')
# # Define the start and end points of the arrow
# arrow_start_x = hyd_n_OH_r_ave + 0.25
# arrow_end_x = 7.5 - 0.25

# # Annotate with the arrow and add text
# ax2.annotate('', xy=(arrow_end_x, hyd_n_OH_ave), xytext=(arrow_start_x, hyd_n_OH_ave), color=red,
#              arrowprops=dict(facecolor=red, edgecolor=red, linewidth=0.75, arrowstyle='-|>', shrinkA=0, shrinkB=0))
# ax2.annotate(r'$n_\text{hyd \ce{OH-}} = \num{5.6}$ ', xy=((arrow_start_x + arrow_end_x) / 2, hyd_n_OH_ave),
#              fontsize=12, color=red, ha='center', va='bottom', textcoords='offset points', xytext=(0, 10))
# ax2.set_ylim(0, 16)
# ax2.set_ylabel(r'hydration hydroxide $n_\text{hyd \ce{OH-}}(r)$ ', color=red)
# ax2.tick_params(axis='y', labelcolor=red, color=red)
# ax2.yaxis.label.set_color(red)
# ax2.spines['right'].set_color(red)

# # Tuckerman
# # axs.scatter(rdf_T_r, rdf_T, marker='*')
# # ax2.scatter(hyd_T_r, hyd_T, marker='*')

# # AIMD
# # data = Post.line_with_errors(AIMD_rdf_F_OHH2O_ave, AIMD_rdf_F_OHH2O_err)
# # axs.plot(AIMD_rdf_F_r, data[0], label='force rdf AIMD')
# # axs.fill_between(AIMD_rdf_F_r, data[1], data[2], alpha=0.25)
# aimd_handle = axs.errorbar(AIMD_rdf_r, AIMD_rdf_OHH2O_ave, yerr=AIMD_rdf_OHH2O_err, fmt='o', markersize=3, label='AIMD', color='C1')

# # MLMD
# data = Post.line_with_errors(rdf_F_OHH2O_ave, rdf_F_OHH2O_err)
# axs.plot(rdf_F_r, data[0], label='rdf MLMD', color='C0')
# mlmd_handle = axs.errorbar(rdf_r, rdf_OHH2O_ave, yerr=rdf_OHH2O_err, fmt='o', markersize=3, label='MLMD', color='C0')

# # Combining legends from both axes in the desired order
# handles = [mlmd_handle, aimd_handle, tip4p_handle, hydration_handle]
# labels = [r'MLMD', r'AIMD', r'MD', r'$n_\text{hyd \ce{OH-}}$']

# # Creating the legend
# axs.legend(handles, labels, loc='best')

# # Adjust layout to prevent overlap
# plt.tight_layout()

# # Saving the figure
# plt.savefig('poster_rdf_OH')
# plt.close()